In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
# Load the Cora dataset (Planetoid is used in the paper)
dataset = Planetoid(root="./data", name="Cora")
data = dataset[0]

Processing...
Done!


In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
        super(GAT, self).__init__()
        # First layer: Multi-head GAT
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        # Second layer: Aggregated multi-head GAT
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.elu(self.gat1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat2(x, edge_index)
        return F.log_softmax(x, dim=1)